<a href="https://colab.research.google.com/github/A0CiE/Assignments/blob/master/ml_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from collections import defaultdict
import os
import tarfile
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

In [ ]:
import shutil

In [ ]:
shutil.rmtree("lingspam_public")

In [ ]:
TRAIN_FOLDER =['part1','part2','part3','part4','part5','part6','part7','part8','part9']
TEST_FOLDER = ['part10']
train_dir = []
test_dir = []
with tarfile.open('lingspam_public.tar.gz', 'r:gz') as tar:
  for tarinfo in tar.getmembers():
    for folder in TRAIN_FOLDER:
      if tarinfo.name.startswith(os.path.join("lingspam_public","lemm_stop",folder,"")):
        train_dir.append(tarinfo.name)
        tar.extract(member=tarinfo)
    for folder in TEST_FOLDER:
      if tarinfo.name.startswith(os.path.join("lingspam_public","lemm_stop",folder,"")):
        test_dir.append(tarinfo.name)
        tar.extract(member=tarinfo)

#1.IG

In [ ]:
def extractWords(msg):
  words = re.findall(r'(?<!\S)[A-Za-z]{2,}(?!\S)', msg)
  for i in range(len(words)):
    words[i] = words[i].lower()
    words[i] = words[i].replace("_", "")
    words[i] = re.sub("[0-9]+", "", words[i])
  words = set(words)
  return words

In [ ]:
bow = set()
spam_count,ham_count = 0,0
spam_words,ham_words = defaultdict(int), defaultdict(int)
for dir in train_dir:
  f = open(dir, "r")
  words = extractWords(f.read())
  if dir.find('spmsg')!=-1:
    spam_count += 1
    for word in words:
      bow.add(word)
      spam_words[word]+=1
  else:
    ham_count+=1
    for word in words:
      bow.add(word)
      ham_words[word]+=1

In [ ]:
p_spam = spam_count / ( spam_count + ham_count )
H_C = - p_spam * np.log2(p_spam) - (1-p_spam) * np.log2(1-p_spam)

h = defaultdict(int)
for word in bow:
  p_0_spam = (1 - spam_words[word] / spam_count) * (p_spam)
  p_1_spam = spam_words[word] / spam_count * p_spam
  p_0_legit = (1 - ham_words[word] / ham_count) * (1 - p_spam)
  p_1_legit = ham_words[word] / ham_count * (1 - p_spam)

  p_x = (spam_words[word] + ham_words[word]) / (spam_count + ham_count)
  if p_0_spam != 0:
    h[word] += p_0_spam * np.log2(p_0_spam / (1 - p_x))
  if p_1_spam != 0:
    h[word] += p_1_spam * np.log2(p_1_spam / p_x)
  if p_0_legit != 0:
    h[word] += p_0_legit * np.log2(p_0_legit / (1 - p_x))
  if p_1_legit != 0:
    h[word] += p_1_legit * np.log2(p_1_legit / p_x)
  h[word] = -h[word]

i_g = defaultdict(int)
for word in bow:
  i_g[word] = H_C - h[word]

#2.Classifier

In [ ]:
def topN(n):
 top_n = sorted(i_g.items(), key=lambda kv:kv[1], reverse=True)[:n]
 return top_n

In [ ]:
def topNfeatureLabel(n, dirs):
  features = {}
  for i, pair in enumerate(topN(n)):
    features[pair[0]] = i;
  data,label = [],[]
  for dir in dirs:
    temp = [0]*n
    f = open(dir, "r")
    words = extractWords(f.read())
    if dir.find('spmsg')!=-1:
      label.append(1)
    else:
      label.append(0)
    for word in words:
      if word in features:
        temp[features[word]] = 1
    data.append(temp)
  return data,label

In [ ]:
top_10 = topN(10)
top_10_words = []
for pair in top_10:
  top_10_words.append(pair[0])

print(top_10_words)

['language', 'remove', 'free', 'linguistic', 'university', 'money', 'click', 'market', 'our', 'business']


In [ ]:
f10_train_data, f10_train_label = topNfeatureLabel(10, train_dir)
f10_train_data = np.array(f10_train_data)
f10_train_label = np.array(f10_train_label)

clf10 = BernoulliNB()
clf10.fit(f10_train_data, f10_train_label)

f10_test_data, f10_test_label = topNfeatureLabel(10, test_dir)
f10_test_data = np.array(f10_test_data)
f10_test_label = np.array(f10_test_label)
pred = clf10.predict(f10_test_data)

# spam precision
true_spam = 0
pred_spam = 0
n = len(pred)
for i in range(n):
  if pred[i] == 1:
    pred_spam += 1
    if f10_test_label[i] == 1:
      true_spam += 1
print('{} true spams out of {} predicted spams. spam precision is {:2.2f}. '.format(true_spam, pred_spam, true_spam / pred_spam))

40 true spams out of 45 predicted spams. spam precision is 0.89. 


In [ ]:
# spam recall
true_spam = 0
spam = 0
for i in range(n):
  if f10_test_label[i] == 1:
    spam += 1
    if pred[i] == 1:
      true_spam += 1
print('{} predicted true spams out of {} all spams. spam recall is {:2.2f}. '.format(true_spam, spam, true_spam / spam))

40 predicted true spams out of 49 all spams. spam recall is 0.82. 


In [ ]:
# correct rate
correct = 0
for i in range(n):
  if pred[i] == f10_test_label[i]:
    correct += 1
print('prediction match rate is {:2.2f}. '.format(correct / n))

prediction match rate is 0.95. 


| Classifier  | #Features  |  Sapm Precision |  Spam recall | 
|---|---|---|---|
|Bernoulli NB classifier with binary features|10  |0.89|0.82|
|Multinomial NB with binary features|10  |   |   |
|Multinomial NB with term frequency (TF) features|10  |   |   |
|---|---|---|---|
|Bernoulli NB classifier with binary features|100 |   |   |
|Multinomial NB with binary features|100 |   |   |
|Multinomial NB with term frequency (TF) features|100 |   |   |
|---|---|---|---|
|Bernoulli NB classifier with binary features|1000|   |   |
|Multinomial NB with binary features|1000|   |   |
|Multinomial NB with term frequency (TF) features|1000|   |   |




#3.SVM

In [ ]:
def topNfeatureLabel(n, dirs):
  features = {}
  for i, pair in enumerate(topN(n)):
    features[pair[0]] = i;
  data,label = [],[]
  for dir in dirs:
    temp = [0]*n
    f = open(dir, "r")
    words = extractWords(f.read())
    if dir.find('spmsg')!=-1:
      label.append(1)
    else:
      label.append(0)
    for word in words:
      if word in features:
        temp[features[word]] = 1
    data.append(temp)
  return data,label

In [ ]:
X, y = topNfeatureLabel(1000, train_dir)
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#clf_svm = svm.SVC(C=1000, kernel='linear')
clf_svm = svm.SVC(C=1000, kernel='rbf')
clf_svm.fit(X_train, y_train)

f1000_test_data, f1000_test_label = topNfeatureLabel(1000, test_dir)
f1000_test_data = np.array(f1000_test_data)
f1000_test_label = np.array(f1000_test_label)
pred_svm = clf_svm.predict(f1000_test_data)

# spam precision
true_spam = 0
pred_spam = 0
n = len(pred_svm)
for i in range(n):
  if pred_svm[i] == 1:
    pred_spam += 1
    if f1000_test_label[i] == 1:
      true_spam += 1
print('{} true spams out of {} predicted spams. spam precision is {:2.2f}. '.format(true_spam, pred_spam, true_spam / pred_spam))

46 true spams out of 47 predicted spams. spam precision is 0.98. 


In [ ]:
# spam recall
true_spam = 0
spam = 0
for i in range(n):
  if f1000_test_label[i] == 1:
    spam += 1
    if pred_svm[i] == 1:
      true_spam += 1
print('{} predicted true spams out of {} all spams. spam recall is {:2.2f}. '.format(true_spam, spam, true_spam / spam))

46 predicted true spams out of 49 all spams. spam recall is 0.94. 


In [ ]:
print('correct prediction rate {:2.2f}. '.format(clf_svm.score(X_test, y_test)))

correct prediction rate 0.99. 


#4.Adversarial Classification

In [ ]:
?